In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

import json
from selenium.webdriver.chrome.service import Service

import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

import re
import datetime
import locale
import numpy as np

from deep_translator import GoogleTranslator


In [2]:
service = Service('C:/Users/alexa/Downloads/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service)
url = 'https://www.greenpeace.fr/guetteur/calendrier/' #mon site à scrapper

driver.get(url) # Ouvrir la page web avec Selenium

# Attendre que le contenu dynamique soit chargé
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "month")))

# stocker le code source de la page dans la variable htlm
html = driver.page_source

# Fermer le navigateur
driver.quit()

In [3]:
soup = BeautifulSoup(html, 'html.parser')
mois_elements = soup.find_all('li', class_='month')
noms_par_mois = {}
for mois in mois_elements:
    nom_du_mois = mois.find('a').get('id') # Récupérer les noms des mois
#     print(nom_du_mois)
    # Le premier 'ul' pour les légumes
    ul_legumes = mois.find_next('ul', attrs={'data-v-a603cee8': True})
    legumes = [li.get_text(strip=True) for li in ul_legumes.find_all('li', attrs={'data-v-a603cee8': True})]
#     print(legumes)
    #  Après celui des légumes, c'est le ul des fruits
    ul_fruits = ul_legumes.find_next('ul', attrs={'data-v-a603cee8': True})
    fruits = []
    if ul_fruits:
        fruits = [li.get_text(strip=True) for li in ul_fruits.find_all('li', attrs={'data-v-a603cee8': True})]
#     print(fruits)    
    # Et le dernier c'est celui des Céréales & légumineuses
    ul_cere = ul_fruits.find_next('ul', attrs={'data-v-a603cee8': True})
    cereales = []
    if ul_cere:
        cereales = [li.get_text(strip=True) for li in ul_cere.find_all('li', attrs={'data-v-a603cee8': True})]
#     print(cereales)
    # j'ajoute dans le dico avec le nom du mois comme clé
    noms_par_mois[nom_du_mois] = {'Legumes': legumes, 'Fruits': fruits, 'Céréales et légumineuses': cereales}
    
    


In [4]:
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')
current_date = datetime.datetime.now()
current_month_name = current_date.strftime("%B")

current_legumes = noms_par_mois[current_month_name]["Legumes"]
current_fruits = noms_par_mois[current_month_name]["Fruits"]
current_cereales = noms_par_mois[current_month_name]["Céréales et légumineuses"]

unique_fruits = set()
unique_legumes = set()
unique_cereales = set()

for mois_info in noms_par_mois.values():
    unique_fruits.update(mois_info.get('Fruits', []))
    unique_legumes.update(mois_info.get('Legumes', []))
    unique_cereales.update(mois_info.get('Céréales et légumineuses', []))
    
unique_fruits_list = list(unique_fruits)
unique_legumes_list = list(unique_legumes)
unique_cereales_list = list(unique_cereales)

In [53]:
translator_en = GoogleTranslator(source='fr', target='en')
def traduire_en(liste):
    return [translator_en.translate(word) for word in liste]

translator_it = GoogleTranslator(source='fr', target='it')

def traduire_it(liste):
    return [translator_it.translate(word) for word in liste]

translator_de = GoogleTranslator(source='fr', target='de')

def traduire_de(liste):
    return [translator_de.translate(word) for word in liste]

In [54]:
current_fruits = current_fruits + traduire_en(current_fruits) + traduire_it(current_fruits) + traduire_de(current_fruits)
current_legumes = current_legumes + traduire_en(current_legumes) + traduire_it(current_legumes) + traduire_de(current_legumes)
current_cereales = current_cereales + traduire_en(current_cereales) + traduire_it(current_cereales) + traduire_de(current_cereales)

unique_fruits_list = unique_fruits_list + traduire_en(unique_fruits_list) + traduire_it(unique_fruits_list) + traduire_de(unique_fruits_list)
unique_legumes_list = unique_legumes_list + traduire_en(unique_legumes_list) + traduire_it(unique_legumes_list) + traduire_de(unique_legumes_list)
unique_cereales_list = unique_cereales_list + traduire_en(unique_cereales_list) + traduire_it(unique_cereales_list) + traduire_de(unique_cereales_list)


In [5]:
import ast
def convert_to_list(string):
    try:
        return ast.literal_eval(string)
    except ValueError:
        return []

In [48]:
paris = pd.read_csv("restaurantsParis.csv")
rome = pd.read_csv("restaurantsRome.csv")
london = pd.read_csv("restaurantsLondon.csv")
berlin = pd.read_csv("restaurantsBerlin.csv")
cardiff = pd.read_csv("restaurantsCardiff.csv")
edinburgh = pd.read_csv("restaurantsEdinburgh.csv")
belfast = pd.read_csv("restaurantsBelfast.csv")


In [58]:
paris["ville"] = "Paris"
rome["ville"] = "Rome"
london["ville"] = "Londres"
berlin["ville"] = "Berlin"
cardiff["ville"] = "Cardiff"
edinburgh["ville"] = "Edinburgh"
belfast["ville"] = "Belfast"

In [59]:
restaurants = pd.concat([paris, rome, london, berlin, cardiff, edinburgh, belfast], ignore_index=True)
restaurants['Menu Items'] = restaurants['Menu Items'].apply(convert_to_list)

In [6]:
restaurants = pd.read_csv("restaurants.csv")
restaurants['Menu Items'] = restaurants['Menu Items'].apply(convert_to_list)

In [ ]:
# translator = Translator()

# unique_fruits_combined = []
# unique_legumes_combined = []
# unique_cereales_combined = []

# for word in unique_fruits_list:
#     french_name = word
#     english_name = translator.translate(word, src='fr', dest='en').text
#     unique_fruits_list.append(english_name)

# for word in unique_legumes_list:
#     french_name = word
#     english_name = translator.translate(word, src='fr', dest='en').text
#     unique_legumes_list.append(english_name)

# for word in unique_cereales_list:
#     french_name = word
#     english_name = translator.translate(word, src='fr', dest='en').text
#     unique_cereales_list.append(english_name)

# for word in current_legumes:
#     french_name = word
#     english_name = translator.translate(word, src='fr', dest='en').text
#     current_legumes.append(english_name)

# for word in current_fruits:
#     french_name = word
#     english_name = translator.translate(word, src='fr', dest='en').text
#     current_fruits.append(english_name)

# for word in current_cereales:
#     french_name = word
#     english_name = translator.translate(word, src='fr', dest='en').text
#     current_cereales.append(english_name)

In [60]:
def getScores(result_string):

    total_fruits_count = 0
    total_legumes_count = 0
    total_cereales_count = 0

    for word in unique_fruits_list:
        total_fruits_count += result_string.lower().count(word.lower())

    for word in unique_legumes_list:
        total_legumes_count += result_string.lower().count(word.lower())

    for word in unique_cereales_list:
        total_cereales_count += result_string.lower().count(word.lower())

        
    if total_fruits_count != 0:
        total_current_fruits_count = 0
        for word in current_fruits:
            total_current_fruits_count += result_string.lower().count(word.lower())
        fruit_score = total_current_fruits_count/total_fruits_count
    else:
        fruit_score = np.nan

    if total_legumes_count != 0:
        total_current_legumes_count = 0
        for word in current_legumes:
            total_current_legumes_count += result_string.lower().count(word.lower())
        legume_score = total_current_legumes_count/total_legumes_count
    else:
        legume_score = np.nan


    
    if total_cereales_count !=0:
        total_current_cereales_count = 0
        for word in current_cereales:
            total_current_cereales_count += result_string.lower().count(word.lower())    
        cereale_score = total_current_cereales_count/total_cereales_count
    else:
        cereale_score = np.nan
    
    
    return fruit_score, legume_score, cereale_score, total_score


for index, row in restaurants.iterrows():
    menu = ' '.join(row["Menu Items"])
    restaurants.loc[index, ['fruit_score', 'legume_score', 'cereale_score', 'total_score']] = getScores(menu)

In [44]:
restaurants.head()

,Restaurant_Name,Ranking,Price_Range,Cuisine_Type,Menu Items,ville,fruit_score,legume_score,cereale_score
0,Café de l'Homme,4.3,51 € et plus,Française,[],NaN,NaN,NaN,NaN
1,Sacré Frenchy !,4.5,De 31 € à 50 €,Française - Traditionnelle,"[accompanied, on, jb, chef, des, meal, ice, bu...",NaN,1.0,1.0,0.0
2,Tracé,4.9,51 € et plus,Française contemporaine,[],NaN,NaN,NaN,NaN
3,"Nellu, Food and Wine Bar",4.8,51 € et plus,Bar à Vin,[],NaN,NaN,NaN,NaN
4,L'Empire du 8ème,4.6,De 31 € à 50 €,Française,"[accompanied, sauteed, on, beefsteak, sauce, o...",NaN,0.0,1.0,0.0


In [ ]:
# restaurants['Menu Items'] = restaurants['Menu Items'].apply(lambda items: ' '.join(items))

In [22]:
for index, row in restaurants.iterrows():
    menu = ' '.join(row["Menu Items"])
    restaurants.loc[index, ['fruit_score', 'legume_score', 'cereale_score', 'total_score']] = getScores(menu)

In [61]:
restaurants

,Restaurant_Name,Ranking,Price_Range,Cuisine_Type,Menu Items,ville,fruit_score,legume_score,cereale_score
0,Café de l'Homme,4.3,51 € et plus,Française,[],Paris,NaN,NaN,NaN
1,Sacré Frenchy !,4.5,De 31 € à 50 €,Française - Traditionnelle,"[accompanied, on, jb, chef, des, meal, ice, bu...",Paris,0.560000,0.739130,0.0
2,Tracé,4.9,51 € et plus,Française contemporaine,[],Paris,NaN,NaN,NaN
3,"Nellu, Food and Wine Bar",4.8,51 € et plus,Bar à Vin,[],Paris,NaN,NaN,NaN
4,L'Empire du 8ème,4.6,De 31 € à 50 €,Française,"[accompanied, sauteed, on, beefsteak, sauce, o...",Paris,0.250000,0.333333,0.0
...,...,...,...,...,...,...,...,...,...
1772,The River Bar Bistro at the Crowne Plaza,3.1,25 £ ou moins,Réservation de tables avec bouteilles,"[bloomer, applewood, clandeboye, northern, yog...",Belfast,0.444444,0.500000,0.0
1773,Binkys Belmont,4.7,25 £ ou moins,Bistro,"[curried, cod, whiskey, yogurt, dijonnaise, su...",Belfast,0.000000,0.470588,0.0
1774,Drawing Office Two,4.1,25 £ ou moins,Irlandaise,"[curried, philly, onions, haddock, apple, caes...",Belfast,0.250000,0.666667,0.0
1775,Binkys,4.3,25 £ ou moins,Café,[],Belfast,NaN,NaN,NaN


In [65]:
restaurants.to_csv('restaurants.csv', index=False)

In [64]:
restaurants = restaurants[restaurants['Menu Items'].apply(lambda x: len(x) > 0)]
restaurants

,Restaurant_Name,Ranking,Price_Range,Cuisine_Type,Menu Items,ville,fruit_score,legume_score,cereale_score
1,Sacré Frenchy !,4.5,De 31 € à 50 €,Française - Traditionnelle,"[accompanied, on, jb, chef, des, meal, ice, bu...",Paris,0.560000,0.739130,0.0
4,L'Empire du 8ème,4.6,De 31 € à 50 €,Française,"[accompanied, sauteed, on, beefsteak, sauce, o...",Paris,0.250000,0.333333,0.0
5,Restaurant Market By Chef Jean Georges,4.6,De 31 € à 50 €,Française,"[roll, fondue, sauce, herbs, ice, jacqueline, ...",Paris,0.250000,0.600000,0.0
6,Le Replat de Montmartre,4.6,30 € ou moins,Européenne,"[sage, sauce, ice, fish, belle, three, la, tri...",Paris,0.500000,0.333333,0.0
7,Pamela Popo,4.8,51 € et plus,Française,"[devilled, sauce, herbs, pickles, bone, la, se...",Paris,0.533333,0.375000,0.0
...,...,...,...,...,...,...,...,...,...
1770,White's,4.2,25 £ ou moins,Irlandaise,"[applewood, cod, fletchers, onions, haddock, b...",Belfast,0.666667,0.875000,0.0
1772,The River Bar Bistro at the Crowne Plaza,3.1,25 £ ou moins,Réservation de tables avec bouteilles,"[bloomer, applewood, clandeboye, northern, yog...",Belfast,0.444444,0.500000,0.0
1773,Binkys Belmont,4.7,25 £ ou moins,Bistro,"[curried, cod, whiskey, yogurt, dijonnaise, su...",Belfast,0.000000,0.470588,0.0
1774,Drawing Office Two,4.1,25 £ ou moins,Irlandaise,"[curried, philly, onions, haddock, apple, caes...",Belfast,0.250000,0.666667,0.0
